## Data Loader & NN making

### DataLoader (Input value setting?)
* 입력은 어떻게 정의해?
* batch나 epoch는 어떻게 설정해?
* 전처리도 직접 하고 싶은데?
* MNIST, CIFAR-10 이런 거 말고 내 데이터를 직접 넣고 싶은데?ㅠㅠ

1. **torchvision**을 이용해 CIFAR-10의 학습, 평가 데이터셋을 로드하여 정규화한다.
2. CNN을 정의한다.
3. 손실 함수를 정의한다.
4. 학습 데이터를 이용해 학습시킨다.
5. 평가 데이터셋을 이용해 평가한다.

## 다양한 Loss함수들 (torch.nn.XXXX)
![%EC%BA%A1%EC%B2%98.PNG](attachment:%EC%BA%A1%EC%B2%98.PNG)

## MNIST 분류

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import numpy as np

import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper Parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST Dataset 
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

train_dataset = torchvision.datasets.MNIST(root='./mnist', 
                            train=True, 
                            download=True,
                            transform=trans)

test_dataset = torchvision.datasets.MNIST(root='./mnist', 
                           train=False, 
                           download=True,
                           transform=trans)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


Processing...

Done!

In [5]:
# Neural Network Model (1 hidden layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes)

In [6]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [7]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = images.view(-1, 28*28)
        labels = labels
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
print('Training Completed')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/5], Step [100/600], Loss: 0.3417

Epoch [1/5], Step [200/600], Loss: 0.2361

Epoch [1/5], Step [300/600], Loss: 0.3143

Epoch [1/5], Step [400/600], Loss: 0.2599

...

Epoch [5/5], Step [300/600], Loss: 0.0654

Epoch [5/5], Step [400/600], Loss: 0.0728

Epoch [5/5], Step [500/600], Loss: 0.1349

Epoch [5/5], Step [600/600], Loss: 0.1041

Training Completed

In [10]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = images.view(-1, 28*28)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %

In [11]:
net

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

## CIFAR-10 CNN 분류

CIFAR-10 데이터셋을 이용할 것이다. 해당 데이터셋은 비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭 순서의 클래스들을 가지고 있다. CIFAR-10의 이미지들은 3 * 32 * 32인데 다시 말해 32 * 32 픽셀 크기의 3채널 컬러 이미지이다.
![%EC%BA%A1%EC%B2%98.PNG](attachment:%EC%BA%A1%EC%B2%98.PNG)

In [13]:
# 불러온 이미지 선처리 하기 위한 단계 (ToTensor : 이미지를 텐서 형태로)
# Tensor의 range는 0에서 1로
# 채널에 변화를 줌(?)
# CenterCrop : 이미지 가운데 부분만 잘라서 사용하겠다
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
# 이미지 전처리를 위한 transforms.Compose
# PIL image (H x W x C) -> Tensor (C x H x W)

trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)

Files already downloaded and verified

In [14]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=4,
                                          shuffle=True, 
                                          num_workers=0)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=0)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [17]:
# 학습 이미지 예시

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images, nrow=4))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

 deer horse plane   car

### 2. CNN 정의

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
net = Net()

### 3. 손실 함수 및 optimizer 정의
손실 함수는 Classfication Cross-Entropy를 사용하고, optimizer는 momentum을 세팅한 SGD를 이용할 것이다.

In [19]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum=0.9)

### 4. 네트워크 학습
단순히 데이터를 반복시켜 네트워크와 optimizer의 입력으로 넘겨주기만 하면 된다.

In [20]:
for epoch in range(2): # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:     # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
print('Finished Training')

[1,  2000] loss: 2.302

[1,  4000] loss: 2.301

[1,  6000] loss: 2.299

[1,  8000] loss: 2.293

[1, 10000] loss: 2.274

...

[2,  6000] loss: 1.934

[2,  8000] loss: 1.883

[2, 10000] loss: 1.846

[2, 12000] loss: 1.788

Finished Training

### 5. 평가 데이터를 이용한 네트워크 평가
학습 데이터셋을 이용해 총 2번 반복하면서 학습을 시켰다. 그러나 실제로 네트워크가 무엇인가를 배웠는지에 대하여 테스트를 해야한다.

뉴럴 네트워크의 출력인 클래스 label을 예측하고 실제 데이터와 비교함으로써 테스트를 수행할 수 있는데 만약 예측 결과가 올바르다면 올바른 예측 리스트에 샘플을 추가할 수 있다.

우선, 첫번째로 이해를 돕기 위해 테스트셋의 이미지를 띄운다.

In [21]:
dataiter = iter(testloader)
images, labels = dataiter.next()

#print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

GroundTruth: 

  cat  ship  ship plane

In [22]:
outputs = net(images)

_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

Predicted: 

  dog  ship  ship  ship

In [23]:
# Test the Model
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
#     images = images.view(-1, 28*28)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 2500 test images: 34 %

## 내 데이터셋을 넣는 방법

In [24]:
# 나의 데이터셋을 넣는 방법
trans2 = transforms.Compose([transforms.Resize((100, 100)), 
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset2 = torchvision.datasets.ImageFolder(root='./dogs', transform=trans2)

In [25]:
len(trainset2)

8

In [29]:
trainloader2 = DataLoader(trainset2, batch_size=8, shuffle=False, num_workers=4)
dataiter2 = iter(trainloader2)
images2, labels2 = dataiter2.next()

In [30]:
print(images2.shape)
imshow(torchvision.utils.make_grid(images2, nrow=4))
print(images2.shape)
print( (torchvision.utils.make_grid(images2)).shape )

torch.Size([8, 3, 100, 100])

torch.Size([8, 3, 100, 100])

torch.Size([3, 104, 818])

## Visdom의 활용
* visdom server 켜기 : anaconda에서 python -m visdom.server => localhost:8097

In [31]:
import visdom
vis = visdom.Visdom()
textwindow = vis.text('Hello, Python')

In [32]:
for i, data in enumerate(trainloader):
    img, label = data
    vis.image(img[0]) # 이미지 하나만 볼 때
    vis.images(img) # 이미지 여러개 묶음을 볼 때
    break

In [33]:
plt = vis.line(Y = torch.randn(5)) # Y 값만 넣어주면 x는 디폴트로 0에서 1
plot = vis.line(Y = torch.randn(5), X = np.array([0,1,2,3,4]))

In [34]:
# plot을 업데이트하기
# plot이라는 이름의 그래프에 x=5에 값이 하나 추가됨
vis.line(Y=torch.randn(1), X=np.array([5]), win=plot, update='append')

'window_368d89b43f0142'

In [35]:
# 계속 업데이트하고 싶으면?
for i in range(500):
    vis.line(Y = torch.randn(1), X = np.array([i+5]), win=plot, update='append')

In [36]:
# 한 플롯에 두개 그리기
# 세 개라면 Y=torch.randn(n,3)으로 적고 x를 세 개의 seperate한 범위 적어주면 됨
vis.line(Y = torch.randn(10,2), X = np.column_stack((np.arange(0,10), np.arange(0,10))))

'window_368d89ca808470'

In [37]:
# plot 형태 변형 및 정보 추가
vis.line(Y = torch.randn(10,2), X = np.column_stack((np.arange(0,10), np.arange(0,10))),
         opts=dict(title='hello',
                   showlegend=True))

vis.line(Y = torch.randn(10,2), X = np.column_stack((np.arange(0,10), np.arange(0,10))),
         opts=dict(title='hello2',
                   legend=['1번', '2번'],
                   showlegend=True))

'window_368d89cef00adc'

[WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다


## 더 많은 내용
* https://github.com/facebookresearch/visdom